# Option-Critic NTMs

## Options

A framework for essentially having sub-policies in certain states (options). More formally introduced in [1,2].

Options can be represented as $\omega \in \Omega$ (option). Where each option is a triple ($I_\omega, \pi_\omega, \beta_\omega$). $I_\omega$ is the set of states where the option can start. $\pi_\omega$ is the policy to follow in the option states. $\beta_\omega$ is the termination function (probability of terminating in each state [3]).

An MDP with a set of options is known as a semi-markov decision process [4] with its own optimal value function and option-value function ($V_\Omega (s)$ and $Q_\Omega (s)$, respectively).

### Learning Options

**Idea** Learn option policies and termination functions (assuming they are differentiable parameterized function approximators) [4]. 

*Call-and-return* option execution model: pick an option ($\omega$) "according to its policy over options ($\pi_\Omega$" [4]. Follow $\pi_\omega$ (the option policy) until a termination state as indicated by $\beta_\omega$.

$\pi_{\omega, \theta}$, $\beta_{\omega, \vartheta}$ are the intra-option policy and termination functions parameterized by $\theta, \vartheta$.

## Option-Critic

Ways of learning options that maximize the expected return in the current task, but extra information can be added to the objective as long as it's differentiable. 

Define the option-value function as:

$$Q_\Omega (s,\omega) = \sum_a \pi_{\omega, \theta} (a|s) Q_U (s,\omega, a)$$

$Q_U$ is "the value of executing an action in the context of a state-action pair" [4]. 

$$Q_U (s, \omega, a) = r(s,a) + \gamma \sum_{s'} P(s' | s,a) U( \omega, s')$$

$U$ is the option-value function *upon arrival*  [4].

$$U(w,s') = (1- \beta_{w,\vartheta} (s')) Q_\Omega (s', w) + \beta_{\omega, \vartheta} (s') V_\Omega (s')$$

Note, above we are essentially saying, continue to use the option-value function if we do not exit, otherwise use the value function over options.

We say that if option $\omega_t$ is ongoing at time $t$ in state $s_t$, then:

$$P(s_{t+1}, \omega_{t+1} | s_t, \omega_t) = \sum_a \pi_{\omega_t, \theta}( a | s_t ) P( s_{t+1} | s_t, a)((1-\beta_{w_t, \vartheta}(s_{t+1})) 1_{\omega_t = \omega_{t+1}}+ \beta_{\omega_t, \vartheta} (s_{t+1}) \pi_\Omega (\omega_{t+1} | s_{t+1}))$$

Then we can take the gradient of the option-value function as:

$$\frac{\partial Q_\Omega (s, \omega)}{\partial \theta} = \left( \sum_a \frac{ \partial \pi_{\omega, \theta} (a| s)}{\partial \theta} Q_U (s, \omega, a) \right) + \sum_a \pi_{\omega, \theta} (a| s) \sum_{s'} \gamma P (s' | s, a) \frac{\partial U (w,s')}{\partial \theta}$$

Similarly,

$$\frac{\partial Q_\Omega (s, \omega)}{\partial \vartheta} = \sum_a \pi_{\omega, \vartheta} (a| s) \sum_{s'} \gamma P (s' | s, a) \frac{\partial U (\omega,s')}{\partial \vartheta}$$

and 

$$\frac{\partial U (\omega,s') }{\partial \vartheta} = - \frac{\partial \beta_{\omega, \vartheta}(s')}{\partial \vartheta} A_\Omega (s', \omega) + \gamma \sum_{\omega'} \sum_{s''} P(s'', \omega' | s', \omega) \frac{\partial U (\omega',s'')}{\partial \vartheta}$$

Where $A_\Omega (s', \omega) = Q_\Omega (s', \omega) - V_\Omega (s')$ is the advantage function.

**Intuitive interpretation** From [4], note that the advantage function is often used as a baseline to reduce variance in gradient estimates, but is derived naturally here. This can be interpreted as follows. "When the option choice is suboptimal w.r.t the expected value over all options, the advantage function is negative and it drives the gradient corrections up, which increases the odds of terminating." And thus can pick a better option using $\pi_\Omega$. 

**Relation to actor-critic** Here, formulate $\beta_\omega$ and $\pi_\omega$ as belonging to the actor, while the critic is $Q_U$ and $A_\Omega$. 

#### A possible option-critic algorithm formulation example from [4]

![option-critic](option-critic-algo.png "Title")

Notice how the equations from above come into play here. We'll take a look at this in code form in a little bit.

#### The general architecture

But the general architecture can be seen in the figure below from [4]. Not we group our actors and critic as aforementioned. The difference is that we calculate the TD error to generate a gradient as in actor-critic and then apply it. We'll see this in the code a little later.

![option-critic](option-critic-arch.png "Title")




### Code as deep things
From https://github.com/jeanharb/option_critic/blob/master/neural_net.py want to walk through the code a bit to make it more clear how things are working in action.

In [ ]:
    self.train_critic = theano.function([], [critic_cost], updates=critic_updates, givens=critic_givens)
    self.train_actor = theano.function([s], [], updates=actor_updates, givens=actor_givens)
    self.pred_score = theano.function([], T.max(Q, axis=1), givens={x:self.x_shared})
    self.sample_termination = theano.function([s], [termination_sample,T.argmax(Q, axis=1)], givens={o:self.o_shared})
    self.sample_options = theano.function([s], T.argmax(Q, axis=1))
    self.sample_actions = theano.function([s], sampled_actions, givens={o:self.o_shared})
    self.get_action_dist = theano.function([s, o], action_probs)
    self.get_s = theano.function([], s, givens={x:self.x_shared})

Let's take a deeper dive. Remember our critic is $Q_U$ and $A_\Omega$, so let's look at the critic updates in the codebase... Explanations will be inline and relevant code will be extracted in parts. Only the core part of the update rules will be extrected.

#### Critic updates as code

In [ ]:

r = T.fvector()
terminal = T.ivector()
next_x = T.ftensor4()
scale = 255. # Divide by 255 to normalize pixel values
o = T.ivector()

# neural network models for state and termination models
self.state_model = Model(state_network, input_size=input_size, dnn_type=dnn_type) # neural network
self.termination_model = Model(termination_network, input_size=output_size, dnn_type=dnn_type)

# Get the states from the state model (i.e. we're making a function approximator for the states which aren't discrete)
s = self.state_model.apply(x/scale)

# basically run the next_x inputs through the state model
next_s = self.state_model.apply(next_x/scale)

# theano.gradient.disconnected_grad ensures that we don't backprob with respect to next_s
# but here we compute \beta(s')
next_termination_probs = self.termination_model.apply(theano.gradient.disconnected_grad(next_s))
# and here we determine the probabilities for the next option
next_option_term_prob = next_termination_probs[T.arange(o.shape[0]), o]

# Assume not Double Q, see update rule above in tabular algorithm
y = r + (1-terminal)*gamma*(
(1-disc_option_term_prob)*next_Q_prime[T.arange(o.shape[0]), o] +
disc_option_term_prob*T.max(next_Q_prime, axis=1))

# make sure not to backprop through this
y = theano.gradient.disconnected_grad(y)

# Q value for options
option_Q = Q[T.arange(o.shape[0]), o]

# Gather the TD errors to get the Q value over options and update it.
td_errors = y - option_Q

# Quadratic cost function with respect to TD errors
quadratic_part = T.minimum(abs(td_errors), clip_delta)
linear_part = abs(td_errors) - quadratic_part
td_cost = 0.5 * quadratic_part ** 2 + clip_delta * linear_part

# Sum the TD cost function to optimize the critic
critic_cost = T.sum(td_cost)

# Specify the gradient parameters
critic_params = self.Q_model.params + self.state_model.params

# This is passed in, RMSProp is what's used in the paper
learning_algo = self.Q_model.get_learning_method(learning_method, **learning_params)

# Take the gradient with respect to the critic params
grads = T.grad(critic_cost, critic_params)

# Apply the gradient
critic_updates = learning_algo.apply(critic_params, grads, grad_clip=grad_clip)

# Shared weights for theano
critic_givens = {x:self.x_shared, o:self.o_shared, r:self.r_shared,
terminal:self.terminal_shared, next_x:self.next_x_shared}

# Theano critic update function
self.train_critic = theano.function([], [critic_cost], updates=critic_updates, givens=critic_givens)

#### Actor updates as Code

In [ ]:
o = T.ivector()

# Get the states from the state model (i.e. we're making a function approximator for the states which aren't discrete)
s = self.state_model.apply(x/scale)

# get the termination probabilities for the sample states based on observations
termination_probs = self.termination_model.apply(theano.gradient.disconnected_grad(s))

# Only get the term probabilities for the options you're intereted in.
option_term_prob = termination_probs[T.arange(o.shape[0]), o]

# actor model parameters (not this is the options model and the termination model part of the network)
actor_params = self.termination_model.params + self.options_model.params

# This is just SGD at a different learning rate than the critic (there's a realation to GANs here)
learning_algo = self.termination_model.get_learning_method("sgd", lr=actor_lr)

# prevent updates to the Q_\Omega
disc_Q = theano.gradient.disconnected_grad(option_Q)

# Prevent updates to V_\Omega
disc_V = theano.gradient.disconnected_grad(T.max(Q, axis=1))

# termination_reg is a constant, seems to be set to 0.0 in the codebase by default
# note how below is the action function with some \epsilon value for regularization.
# In the paper described as 0.01 termination regularization,
term_grad = T.sum(option_term_prob*(disc_Q-disc_V+termination_reg))

# get the action probabilities from the function approximator 
action_probs = self.options_model.apply(s, o)

# Get the entropy for the action approximations
entropy = -T.sum(action_probs*T.log(action_probs))

# in the codebase by default do not have a baseline, but in the paper, they use a baseline and entropy_reg of 0.01
if not BASELINE:
  policy_grad = -T.sum(T.log(action_probs[T.arange(a.shape[0]), a]) * y) - entropy_reg*entropy
else:
  policy_grad = -T.sum(T.log(action_probs[T.arange(a.shape[0]), a]) * (y-disc_Q)) - entropy_reg*entropy

# Get our policy gradient
grads = T.grad(term_grad+policy_grad, actor_params)

# get our actor updates
actor_updates = learning_algo.apply(actor_params, grads, grad_clip=grad_clip)

# Shared weights for theano
actor_givens = {a:self.a_shared, r:self.r_shared,
terminal:self.terminal_shared, o:self.o_shared, next_x:self.next_x_shared}

# actor updates
self.train_actor = theano.function([s], [], updates=actor_updates, givens=actor_givens)


#### Outline for learning (extracted code)


In [ ]:
new_option = self.model.predict_move(s)[0]
current_option = np.random.randint(self.params.num_options) if np.random.rand() < epsilon else new_option

while not game_over:
    # get the current action
    current_action = self.model.get_action(s, [current_option])[0]
    
    # sample it
    reward, raw_reward, new_frame = self.act(current_action, testing=testing)
    
    # Get the observations
    x = get_new_frame(new_frame, x)
    
    # Get the function approx for the state
    s = self.model.get_state([x])
    
    # add to dataset for experience replay
    data_set.add_sample(x[-1], current_option, reward, game_over or life_death)
    
    # predict whether we should choose a new option and what option to choose
    term_out = self.model.predict_termination(s, [current_option])
    termination, new_option = term_out[0][0], term_out[1][0]
    
    # choose new option or random option with some epsilon probability
    if self.frame_count < self.params.replay_start_size or termination:
        current_option = np.random.randint(self.params.num_options) if np.random.rand() < epsilon else new_option

    total_reward += raw_reward
    
    # train the actor critic networks
    if self.frame_count > self.params.replay_start_size:
        self.learn_actor(old_s,
                     np.array(x).reshape(1,4,84,84),
                     [current_option],
                     [current_action],
                     [reward],
                     [game_over or life_death])
    if self.frame_count % self.params.update_frequency == 0:
        self.learn_critic()
    if self.frame_count % self.params.freeze_interval == 0:
        self.model.update_target_params()

## Neural Turing Machines

The foundational principle of neural machines is similar to working memory in humans. RNNs are shown to be Turing Complete, so can we make a differentiable turing machine from them [5]. Using an LSTM as a controller, we can use fthe formulation in the figure below from [5]. This essentially allows us to create read and write heads which address into some memory space in a totally learned/differentiable manner.

![rl-ntm](ntm-interface.png "Title")

### Read operations

We first take a look at how a parameterized differentiable read operation can be formed.

$M_t$ is an $N \times M$ matrix of memory at time $t$

So then we have a learned weighting $w_t$

$$\sum_i w_t (i) = 1, 0 \le w_t(i) \le 1, \forall i$$

and 

$$r_t \leftarrow \sum_i w_t (i) M_t(i)$$

### Write operations

Now let's take a look at a write. In the original NTM formulation, this involves an erase and an addition.

$$\tilde{M}_t (i) \leftarrow M_{t-1}(i) [ 1- w_t(i) e_t]$$
 
$$M_t (i) \leftarrow \tilde{M}_{t}(i) [w_t(i) a_t]$$

That is we erase the weighted units given by an write head $w_t$ and an erase vector $e_t$ and then add and addition vector $a_t$. These vectors allow fine-grained control over which elements are erased and added over.

### Addressing

From [5], we can take a look at the addressing scheme as seen in the figure below. 

![rl-ntm](ntm-addressing.png "Title")

First focus by content (produce a key vector $k_t$ of length $M$ and generate content based on a similarity measure 

$$w_t^C (i) \leftarrow \frac{\exp(\beta_t K[k_t, M_t(i)])}{\sum_j \exp(\beta_t K[k_t, M_t(j)])}$$

Where $K[u,v] = \frac{u \cdot v}{||u|| \cdot ||v||}$

Then we take this content vector and emit an interpolation gate $g_t$

$$w_t^g \leftarrow g_t w_t^c + (1-g_t) w_{t-1}$$

And then we emit a distribution over allowed shifts:

$$\tilde{w}_t(i) \leftarrow \sum_{j=0}^{N-1} w_t^g(j) s_t(i-j)$$

Finally we emit a sharpening parameter

$$w_t(i) \leftarrow \frac{\tilde{w}_t (i)^{\gamma_t}}{\sum_j \tilde{w}_t(j)^{\gamma_t}}$$

This gives us our final weight vector which we use in our read/write operations above. The combination of these machines optimized over the sequence loss create a totally differentiable system that can learn to manipulate memory.

## Neural Turing Computers in the Context of RL

### Differentiable Neural Computers

This is the follow-up work on NTMs, allows for deallocation of memory and allows re-use of memory cells (adds free gates) [6]. Can be trained with reinforcement learning techniques.

In the work they describe how: "the architecture of the reinforcement learning agent presented here contains two DNC networks: a policy network that selects an action and a value network that estimates the expected future reward given the policy network and current state."

The "policy-network DNC observes the environment states by receiving an observation sequence  $o_1, o_2, \dots, o_t$, one step at a time and conditions its action on the sequence seen so far: $\pi( \cdot |   o_1, \dots , o_t; \theta)$. The value-network DNC tries to predict the sum of future rewards for this policy given the current history of observations" [6].

"The value network learns by supervised regression to predict the sum of future rewards. After a mini-batch of $L$ episodes, the value network updates its parameters $\phi$ using gradient descent on the loss function" [6]: $\frac{1}{2L} \sum_{l=1}^L \sum_{t=1}^T ||\sum_{\tau=1}^T r(s_\tau^l, a_\tau^l) - V^\pi(o_1, \dots, o_\tau; \phi)$

"The action distribution of the policy network $\pi(a_t |   o_1, \dots, o_t; \theta)$ is a softmax over a discrete set of actions. We use a policy gradient method to optimize the policy parameters4. After each mini-batch of $L$ episodes, the policy parameter gradient direction to ascend is:" [6]

![rl-ntm](dnc-functionn.png "Title")

Note that the advantage function is the second term. Starting to see some similarity here with option-critic?

### Briefly on RL-NTMs

An alternative to NTMs are RL-NTMs [8]. These are neural turing machines based on reinforcement learning directly. The authors say that it is fairly unstable and complex. The idea is fairly simple, model controller a discrete way, but this makes it non-differentiable. Hence, you have to use REINFORCE on the objective. Some figures below make the idea more clearly, but we won't go into too much detail here on it. All figures in the section below are from [8].

![rl-ntm](rl-ntm-objective.png "Title")

The interface can thus be described in the Figures.

![rl-ntm](rl-ntm-interface.png "Title")

Above we see the general layout for the architecture, and below we see the optimization methods.

![rl-ntm](rl-ntm-interfaces.png "Title")


### Tricks used to make RL with NTMs Work

#### Curriculum Learning

With both DNCs and RL-NTMs, the authors were unable to successfully learn tasks without curriculum learning. The idea behind curriculum learning is to show steadily more complex tasks based on some parameter. 

#### Experience Replay (through curriculum learning)

Another important part of curriculum learning and the RL formulations is experience replay. Not only is experience replay needed as in [7], but as part of curriculum learning needs to incorporate uniformly distrubted batches from past "lessons" in the curriculum to prevent loss of knowledge. See [6,8] for more information on this.

### Similar Differentiable Memory Machines

There are several works which are based on the idea of augmenting networks with memory including [9-13]. These include varitions on NTMs (Hierarchical, Continuous space, etc.) and variations on Memory Networks used for question answering. As well, there are expansions on adding structured memory to deep reinforcement learning.


## A Proposal for Option-Critic NTMs

There are several different thought processes when one can take when providing a connection between NTMs and Option-Critic. One idea is to use the options as ways to address into and write to memory. Another is to simply use DNCs or NTMs as approximators or policies. We will address the latter first as it is a much more straightforward formulation based on existing work done as part of [6].

### Options from NTMs and DNCs

In the formulation for Differential Neural Computers, the authors proposed a way to use reinforcement learning in conjunction with the DNC (essentially a modified NTM). We describe this above, note how similar these methods are to the option-critic method described earlier. If we now use a DNC to approximate the option policies instead of just the policy-network, then we reach quickly the same formulation. The figure from the option-critic paper describes how this is already done with a neural network [4].

![rl-ntm](option-critic-network.png "Title")

This formulation is fairly simple and intuitive, as in the DNC paper [6] we can simply formulate the RL problem in the same manner, but using the option-critic framework instead of simple policy-gradient methods without options. 

### Options as addressing mechanisms

Another clear correlation between options and neural turing machines that is perhaps more interesting to examine is the relation of options to addressing. Let us take our option-critic architecture and consider it as the controller for our read-write heads. We can consider that the $I,\beta$ functions will be our formulations for the read/write policy to a given memory space. That is, it will control where to write/read from given an input feature. Then we can take our $\pi_\omega$ values to be the read/write policy for that space and $\pi_\Omega$ to as the policy over options (that is which space to write to given $I,\beta, \pi_\omega$).

Intuitively now, our policy over options becomes our addressing mechanism while our policies become our tuning functions within a given "memory slot" of how to store data in that slot. We can define actions as reading or writing some data to a memory slot. For simplicity, we can say for now that the action space can be the size of the memory (but this can likely be loosened with an approximation, such that an action maps to a learned soft-addressing).

The state space remains as the observed outputs. For example, in the context of a copy-task the state space may be the currently displayed values on the tape. We formulate this such that the reward is equivalent to matching the sequence of states desired or being close enough to the goal state (for example, amount of data copied, or in the context of a sorting task the amount of items in the correct order with a time penalty). 

We can then learn the policies in the exact same manner as option critic, but now we have a large number of actions. As such, the options here equare to our addressing scheme in NTMs, a given option relates to a given state space and will likely match to a set of given read/write action locations for that state space. In this context, we would expect more options to mean a more fine-grained addressing into the memory space as it would handle a smaller subset of states. 

#### An alternative formulation

Another possible alternative formulation is setting our state space to be the memory space. In this formulation we can imagine that the options are more explicitely addressing into memory space, particularly with $I, \beta$. The issue here is finding a way to generate a reward for a given action and state. If we say that a given action can be to read/write/or move +/- 1, this formulation starts to bear some resemblance to RL-NTMs (very roughly). Then each option would directly translate to writing or reading from a subset of its states covered between $I, \beta$. However, the trick becomes to formulate a reward for state, action pairs. Particularly, as the state here directly translated to memory address locations. This may mean that we have to add another variable to the formulation: an emission $e$. We can then say that a given state, action pair is not enough to describe the situation, but state action pair gives an emission which can then be used by the reward. That is reading from a memory slot may give an emission of whatever exists in that memory slot, while moving or writing will give no such emission. In this way we can compare emissions similarly how we compared states in the previous formulation.

### Going forward

I plan on implementing and experimenting with this in the coming months (though perhaps reformulating). If you're interested in collaborating or discussing, email me or let's talk! [firstname].[lastname]@mail.mcgill.ca

## References

[1] Sutton, Richard S., Doina Precup, and Satinder Singh. "Between MDPs and semi-MDPs: A framework for temporal abstraction in reinforcement learning." Artificial intelligence 112.1-2 (1999): 181-211.

[2] Stolle, Martin, and Doina Precup. "Learning options in reinforcement learning." International Symposium on Abstraction, Reformulation, and Approximation. Springer Berlin Heidelberg, 2002.

[3] http://www-anw.cs.umass.edu/~barto/courses/cs687/simsek-lecture2.pdf

[4] Bacon, Pierre-Luc, Jean Harb, and Doina Precup. "The option-critic architecture." arXiv preprint arXiv:1609.05140 (2016).

[5] Graves, Alex, Greg Wayne, and Ivo Danihelka. "Neural turing machines." arXiv preprint arXiv:1410.5401 (2014).

[6] Graves, Alex, et al. "Hybrid computing using a neural network with dynamic external memory." Nature 538.7626 (2016): 471-476.

[7] Schaul, Tom, et al. "Prioritized experience replay." arXiv preprint arXiv:1511.05952 (2015).

[8] Zaremba, Wojciech, and Ilya Sutskever. "Reinforcement learning neural Turing machines." arXiv preprint arXiv:1505.00521 362 (2015).

[9] Parisotto, Emilio, and Ruslan Salakhutdinov. "Neural Map: Structured Memory for Deep Reinforcement Learning." arXiv preprint arXiv:1702.08360 (2017).

[10] Chandar, Sarath, et al. "Hierarchical Memory Networks." arXiv preprint arXiv:1605.07427 (2016).

[11] "Dynamic Memory Networks for Natural Language Processing." Kumar et al. arXiv Pre-Print (2015).

[12] Gulcehre, Caglar, et al. "Dynamic Neural Turing Machine with Soft and Hard Addressing Schemes." arXiv preprint arXiv:1607.00036 (2016).

[13] Weston, Jason, Sumit Chopra, and Antoine Bordes. "Memory networks." arXiv preprint arXiv:1410.3916 (2014).
